# Agentive Chatbot

In [5]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from openai import OpenAI
import yaml

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")

#### Variables to store the default system prompt and the catch-all prompt (to be used when no route is identified)

In [6]:
with open('syst_prompt.txt', 'r') as f:
    system_prompt = f.read()

with open('scenario_prompt.txt', 'r') as f:
    scenario_prompt = f.read()

In [7]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []
    
    def query(self, query: str) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": self.role + scenario_prompt})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                if isinstance(conversation, str):
                    file.write(conversation + "\n")
                elif isinstance(conversation, list):
                    for message in conversation:
                        if message["role"] == "user":
                            file.write("User: " + message["content"] + "\n")
                        elif message["role"] == "assistant":
                            file.write("Agent: " + message["content"] + "\n")

rpa = ChatBot(api_key=api_key, role = system_prompt)

In [ ]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value
    rpa.query(pasted_content)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

# Additional input field for pasted content
pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')  # Adjust height as needed
)
# Button for pasted content submission
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'  # Change to your desired color

# Button for saving chat history
save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'  # Change to your desired color

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

# Arrange buttons side by side and center them using HBox
button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

# Display the input fields and the button layout
display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')

Hello! How can I assist you today?

